# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770337344083                   -0.53    9.0         
  2   -2.771688508452       -2.87       -1.30    1.0    104ms
  3   -2.771714462051       -4.59       -2.61    1.0    122ms
  4   -2.771714696867       -6.63       -3.75    1.0    131ms
  5   -2.771714714508       -7.75       -4.12    2.0    106ms
  6   -2.771714715229       -9.14       -5.47    1.0   93.9ms
  7   -2.771714715249      -10.70       -5.71    2.0    127ms
  8   -2.771714715250      -12.43       -6.15    2.0    111ms
  9   -2.771714715250      -14.88       -6.71    1.0    118ms
 10   -2.771714715250      -15.35       -7.15    1.0    100ms
 11   -2.771714715250      -15.05       -8.06    2.0    123ms


-0.0001345745033762076

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770516431421                   -0.52    9.0         
  2   -2.771779655160       -2.90       -1.32    1.0   88.4ms
  3   -2.771801650751       -4.66       -2.45    1.0    121ms
  4   -2.771801996949       -6.46       -3.15    1.0   91.2ms
  5   -2.771802074109       -7.11       -4.12    2.0    106ms
  6   -2.771802074257       -9.83       -4.27    1.0    128ms
  7   -2.771802074465       -9.68       -4.99    1.0   96.2ms
  8   -2.771802074473      -11.07       -5.26    1.0    100ms
  9   -2.771802074476      -11.54       -6.23    1.0    114ms
 10   -2.771802074476      -13.08       -6.33    2.0    124ms
 11   -2.771802074476      -13.51       -6.90    1.0    112ms
 12   -2.771802074476      -14.57       -7.90    2.0    130ms
 13   -2.771802074476   +  -14.10       -7.79    2.0    127ms
 14   -2.771802074476   +  -14.35       -8.46    1.0    131ms


0.01761222046123023

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.0001345745033762076
Displaced dipole:  0.01761222046123023
Polarizability :   1.7746794964606436


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920627226e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551350769e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852771286490e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694593787110e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088787529503e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.274321799993e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 6.885692088061e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 6.885692088061e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.587139416407e-09
[ Info: GMRES linsolve in iter 2; step 2: normres = 3.359238697591e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 4.845752750244e-12
[ Info: GMRES linsolve in iter 2; step 4: normres = 6.188675348030e-14
[ Info: GMRES linsolve in iter 2; finished at step 4: normres = 6.188675348030e-14
┌ Info: GMRES linsolve converged at iteration 2, step

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.